In [1]:
import trimesh
import numpy as np
from scipy.spatial import KDTree

def calculate_volume_difference(mesh1, mesh2):
    """
    Calculate the absolute difference in volumes between two meshes.
    """
    volume1 = mesh1.volume
    volume2 = mesh2.volume
    return abs(volume1 - volume2)

def calculate_iou(mesh1, mesh2):
    """
    Calculate the Intersection over Union (IoU) of two meshes using their volumes.
    """
    intersection_mesh = trimesh.boolean.intersection([mesh1, mesh2], engine='vtk')
    union_mesh = trimesh.boolean.union([mesh1, mesh2], engine='vtk')
    
    if intersection_mesh is None or union_mesh is None:
        return 0  # Return 0 if boolean operations fail
    
    intersection_volume = intersection_mesh.volume
    union_volume = union_mesh.volume
    
    return intersection_volume / union_volume if union_volume > 0 else 0

def calculate_hausdorff_distance(mesh1, mesh2):
    """
    Compute the Hausdorff distance between two meshes.

    Parameters:
        mesh1_vertices (np.ndarray): Vertices of the first mesh as a (N, 3) array.
        mesh2_vertices (np.ndarray): Vertices of the second mesh as a (M, 3) array.

    Returns:
        float: The Hausdorff distance between the two meshes.
    """
    mesh1_vertices = np.array(mesh1.vertices)
    mesh2_vertices = np.array(mesh2.vertices)
    def compute_distance(source, target):
        tree = KDTree(target)
        distances, _ = tree.query(source)
        return distances


    # Compute distances from mesh1 to mesh2 and vice versa
    dist1 = compute_distance(mesh1_vertices, mesh2_vertices)
    dist2 = compute_distance(mesh2_vertices, mesh1_vertices)

    # Hausdorff distance is the maximum distance
    return max(np.max(dist1), np.max(dist2))

def calculate_mean_vertex_distance(mesh1, mesh2):
    """
    Compute the Hausdorff distance between two meshes.

    Parameters:
    mesh1_vertices (np.ndarray): Vertices of the first mesh as a (N, 3) array.
    mesh2_vertices (np.ndarray): Vertices of the second mesh as a (M, 3) array.

    Returns:
    float: The Hausdorff distance between the two meshes.
    """

    mesh1_vertices = np.array(mesh1.vertices)
    mesh2_vertices = np.array(mesh2.vertices)

    def compute_distance(source, target):
        tree = KDTree(target)
        distances, _ = tree.query(source)
        return distances


    # Compute distances from mesh1 to mesh2 and vice versa
    dist1 = compute_distance(mesh1_vertices, mesh2_vertices)
    dist2 = compute_distance(mesh2_vertices, mesh1_vertices)
    # Calculate the average of all distances
    avg_dist1 = np.mean(dist1)
    avg_dist2 = np.mean(dist2)

    # Return the average of the two average distances
    return (avg_dist1 + avg_dist2) / 2

def compare_meshes(mesh1_path, mesh2_path, mesh3_path):
    """
    Load three meshes and compare them using various metrics.
    """
    # Load meshes
    mesh1 = trimesh.load(mesh1_path)
    mesh2 = trimesh.load(mesh2_path)
    mesh3 = trimesh.load(mesh3_path)
    print(f"Mesh 1 volume: {mesh1.volume}")
    print(f"Mesh 2 volume: {mesh2.volume}")
    print(f"Mesh 3 volume: {mesh3.volume}")
    print()
    results = {}
    for pair in [(mesh1, mesh2, "Mesh1-Mesh2"), (mesh1, mesh3, "Mesh1-Mesh3"), (mesh2, mesh3, "Mesh2-Mesh3")]:
        mesh_a, mesh_b, label = pair
        results[label] = {
            "Volume Difference": calculate_volume_difference(mesh_a, mesh_b),
#            "IoU": calculate_iou(mesh_a, mesh_b),
            "Hausdorff Distance": calculate_hausdorff_distance(mesh_a, mesh_b),
            "Mean Vertex Distance": calculate_mean_vertex_distance(mesh_a, mesh_b)
        }
    
    return results

# Example usage
if __name__ == "__main__":
    # Replace these paths with your mesh file paths
    mesh1_path = "/Users/bogdanmatache/Documents/GitHub/2024-25ab-fai3-specialisation-project-team-specifix-1/presentation_material/bone_ground_truth.stl"
    mesh2_path = "/Users/bogdanmatache/Documents/GitHub/2024-25ab-fai3-specialisation-project-team-specifix-1/presentation_material/bone_ground_truth2.stl"
    mesh3_path = "/Users/bogdanmatache/Documents/GitHub/2024-25ab-fai3-specialisation-project-team-specifix-1/presentation_material/bone_ground_truth3.stl"

    metrics = compare_meshes(mesh1_path, mesh2_path, mesh3_path)
    for pair, metrics_values in metrics.items():
        print(f"Metrics for {pair}:")
        for metric, value in metrics_values.items():
            print(f"  {metric}: {value}")
            print()


Mesh 1 volume: 3.6640742771464168
Mesh 2 volume: 4.634643338479866
Mesh 3 volume: 4.596053290213745

Metrics for Mesh1-Mesh2:
  Volume Difference: 0.9705690613334492

  Hausdorff Distance: 0.47134896230641377

  Mean Vertex Distance: 0.04917426101704194

Metrics for Mesh1-Mesh3:
  Volume Difference: 0.9319790130673282

  Hausdorff Distance: 0.6020464669245413

  Mean Vertex Distance: 0.07719652745850414

Metrics for Mesh2-Mesh3:
  Volume Difference: 0.038590048266121

  Hausdorff Distance: 0.4056343628754685

  Mean Vertex Distance: 0.044426089956958056

